In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt;
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
dfsongs = pd.read_csv("10000.txt", sep="\t", nrows=10000, names=['user', 'song', 'play_count'], header=None)
dfsongs.head(5)

,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [ ]:
from collections import defaultdict
usersDict = defaultdict(int)
with open("10000.txt", "r") as songFile:
  for record in songFile:
    #Fetch user id as the first item from a tab delimited line of record
    user = record.split("\t")[0]
    #Fetch song play count as the last item from a tab delimited line of record
    play_count = int(record.split("\t")[2])
    #Update song play count for a user
    usersDict[user] += play_count
        
userPlayCountList = [{'user': u, 'play_count': p} for u,p in usersDict.items()]
dfSongsPlayCount = pd.DataFrame(userPlayCountList)
#Arrange the user songs played records in descending order to facilitate further analysis
dfSongsPlayCount = dfSongsPlayCount.sort_values(by = 'play_count', ascending = False)
dfSongsPlayCount.to_csv("songs_play_count.csv", index=None)

In [ ]:
totalPlayCount = sum(dfSongsPlayCount.play_count)
(float(dfSongsPlayCount.head(n=100000).play_count.sum())/totalPlayCount)*100
dfUserSubset = dfSongsPlayCount.head(n=100000)

In [ ]:
songsDict = defaultdict(int)
with open("10000.txt", "r") as songFile:
  for record in songFile:
        #Fetch song id as the first item from a tab delimited line of record
    song = record.split("\t")[1]
        #Fetch song play count as the last item from a tab delimited line of record
    play_count = int(record.split("\t")[2])
        #Update song play count for a song
    songsDict[song] += play_count
        
songPlayCountList = [{'song': s, 'play_count': p} for s,p in songsDict.items()]
dfSongsPlaySummary = pd.DataFrame(songPlayCountList)
#Arrange the user songs played records in descending order to facilitate further analysis
dfSongsPlaySummary = dfSongsPlaySummary.sort_values(by = 'play_count', ascending = False)
dfSongsPlaySummary.to_csv("songs_summary.csv", index=None)

In [ ]:
(float(dfSongsPlaySummary.head(n=30000).play_count.sum())/totalPlayCount)*100

100.0

In [ ]:
dfSongPlayCountSubset = dfSongsPlaySummary.head(n=30000)

In [ ]:
dfAllSongs = pd.read_csv("10000.txt", sep="\t", names=['user', 'song', 'play_count'], header=None)
#Filter records for top 40% users using the users subset found earlier
dfSongsTopUsers = dfAllSongs[dfAllSongs.user.isin(list(dfUserSubset.user))]
dfSongsTopSongs = dfSongsTopUsers[dfSongsTopUsers.song.isin(list(dfSongPlayCountSubset.song))]

In [ ]:
#Save the top 30 songs dataset to a file
dfSongsTopSongs.to_csv("top_Songs_Subset.csv", index=False)
#Find the number of records in this subset
dfSongsTopSongs.shape

(946218, 3)

In [ ]:
del dfAllSongs
del dfSongsTopUsers

In [ ]:
dfSongsTopSongs.head(5)

,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [ ]:
dfTrackMetadata_sub = pd.read_csv("song_data.csv")
dfTrackMetadata_sub.head(5)

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [ ]:
dfTrackMetadata_sub.drop_duplicates(['song_id'], inplace=True)
dfPopularSongMetaDataMerged = pd.merge(dfSongsTopSongs, dfTrackMetadata_sub, how="left", 
                                      left_on="song", right_on = "song_id")
dfPopularSongMetaDataMerged = dfPopularSongMetaDataMerged[['user', 'song', 'title', 'play_count', 'release', 'artist_name', 'year']]
dfPopularSongMetaDataMerged.head()

,user,song,title,play_count,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,The Cove,1,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,Entre Dos Aguas,2,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,Stronger,1,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,Constellations,1,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,Learn To Fly,1,There Is Nothing Left To Lose,Foo Fighters,1999


In [ ]:
def create_popularity_recommendation(songData, user, song):
  #Get a count of users for each unique song as recommendation score
  songDataGrouped = songData.groupby([song]).agg({user: 'count'}).reset_index()
  songDataGrouped.rename(columns = {user: 'score'},inplace=True)
    
  #Sort the songs based upon recommendation score
  songDataSorted = songDataGrouped.sort_values(['score', song], ascending = [0,1])
    
  #Generate a recommendation rank based upon score
  songDataSorted['Rank'] = songDataSorted['score'].rank(ascending=0, method='first')
        
  #Get the top 10 recommendations
  popularityBasedRecommendations = songDataSorted.head(10)
  return popularityBasedRecommendations

In [ ]:
recommendations = create_popularity_recommendation(dfPopularSongMetaDataMerged,'user','title')
recommendations

,title,score,Rank
6836,Sehr kosmisch,3947,1.0
8725,Undo,3393,2.0
1964,Dog Days Are Over (Radio Edit),3359,3.0
9496,You're The One,3190,4.0
6498,Revelry,2883,5.0
6825,Secrets,2761,6.0
3437,Horn Concerto No. 4 in E flat K495: II. Romanc...,2565,7.0
2595,Fireflies,2303,8.0
3322,Hey_ Soul Sister,2247,9.0
8494,Tive Sim,2194,10.0


In [ ]:
#Aggregate by user and calculate sum of all songs listened by an user
dfPopularSongMetaDataMergedSum  = dfPopularSongMetaDataMerged[['user','play_count']].groupby('user').sum().reset_index()
dfPopularSongMetaDataMergedSum.rename(columns={'play_count':'total_play_count'},inplace=True)
dfPopularSongMetaDataMerged = pd.merge(dfPopularSongMetaDataMerged,dfPopularSongMetaDataMergedSum)
#Calculate the fractional play count
dfPopularSongMetaDataMerged['fractional_play_count'] = dfPopularSongMetaDataMerged['play_count']/dfPopularSongMetaDataMerged['total_play_count']

In [ ]:
#Examine the structure of the newly formed dataset
dfPopularSongMetaDataMerged.head()

,user,song,title,play_count,release,artist_name,year,total_play_count,fractional_play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,The Cove,1,Thicker Than Water,Jack Johnson,0,71,0.014085
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,Entre Dos Aguas,2,Flamenco Para Niños,Paco De Lucia,1976,71,0.028169
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,Stronger,1,Graduation,Kanye West,2007,71,0.014085
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,Constellations,1,In Between Dreams,Jack Johnson,2005,71,0.014085
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,Learn To Fly,1,There Is Nothing Left To Lose,Foo Fighters,1999,71,0.014085


In [ ]:
from scipy.sparse import coo_matrix

small_set = dfPopularSongMetaDataMerged

#Convert user and song IDs to integer values that can be handled by numpy matices
user_codes = small_set.user.drop_duplicates().reset_index()
song_codes = small_set.song.drop_duplicates().reset_index()
user_codes.rename(columns={'index':'user_index'}, inplace=True)
song_codes.rename(columns={'index':'song_index'}, inplace=True)
song_codes['so_index_value'] = list(song_codes.index)
user_codes['us_index_value'] = list(user_codes.index)
small_set = pd.merge(small_set,song_codes,how='left')
small_set = pd.merge(small_set,user_codes,how='left')
mat_candidate = small_set[['us_index_value','so_index_value','fractional_play_count']]
data_array = mat_candidate.fractional_play_count.values
row_array = mat_candidate.us_index_value.values
col_array = mat_candidate.so_index_value.values

#Create a sparse matrix
data_sparse = coo_matrix((data_array, (row_array, col_array)),dtype=float)
data_sparse

<35145x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 946218 stored elements in COOrdinate format>

In [ ]:
pip install importlib

In [ ]:
from scipy.linalg import svd

In [ ]:
import math as mt
import numpy as np
from scipy.sparse.linalg import *
from scipy.sparse.linalg import svds
from scipy.sparse import csc_matrix
import numpy as np


def compute_svd(urm, K):
  U, s, Vt = svds(urm, K)

  dim = (len(s), len(s))
  S = np.zeros(dim, dtype=np.float32)
  for i in range(0, len(s)):
    S[i,i] = mt.sqrt(s[i])

  U = csc_matrix(U, dtype=np.float32)
  S = csc_matrix(S, dtype=np.float32)
  Vt = csc_matrix(Vt, dtype=np.float32)
    
  return U, S, Vt

def compute_estimated_matrix(urm, U, S, Vt, uTest, K, MAX_PID, MAX_UID,test):
  rightTerm = S*Vt 
  max_recommendation = 250
  estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
  recomendRatings = np.zeros(shape=(MAX_UID,max_recommendation ), dtype=np.float16)
  for userTest in uTest:
    prod = U[userTest, :]*rightTerm
    estimatedRatings[userTest, :] = prod.todense()
    recomendRatings[userTest, :] = (-estimatedRatings[userTest, :]).argsort()[:max_recommendation]
  return recomendRatings

In [ ]:
import importlib
importlib.reload(SVD_Helper)

In [ ]:
import SVD_Helper

K=50
utilityMatrix = data_sparse
MAX_PID = utilityMatrix.shape[1]
MAX_UID = utilityMatrix.shape[0]

U, S, Vt = SVD_Helper.compute_svd(utilityMatrix, K)

In [ ]:
uTest = [2, 10, 15, 30, 115, 200]

uTest_recommended_items = SVD_Helper.compute_estimated_matrix(utilityMatrix, U, S, Vt, uTest, K, MAX_PID, MAX_UID, True)

for  user  in  uTest:
    print("\n-------------------------------------- \
    \nRecommendation for user with id: {}\n--------------------------------------\n". format(user))
    rank_value = 1
    for i in uTest_recommended_items[user,0:10]:
        song_details = small_set[small_set.so_index_value == i].drop_duplicates('so_index_value')[['title','artist_name']]
        print("#{}: {} By {}".format(rank_value, list(song_details['title'])[0],list(song_details['artist_name'])[0]))
        rank_value+=1


--------------------------------------     
Recommendation for user with id: 2
--------------------------------------

#1: Home By Edward Sharpe & The Magnetic Zeros
#2: Canada By Five Iron Frenzy
#3: Behind The Sea [Live In Chicago] By Panic At The Disco
#4: Clocks By Coldplay
#5: Creep (Explicit) By Radiohead
#6: Yellow By Coldplay
#7: The Scientist By Coldplay
#8: Pursuit Of Happiness (nightmare) By Kid Cudi / MGMT / Ratatat
#9: Dead Souls By Nine Inch Nails
#10: West One (Shine On Me) By The Ruts

--------------------------------------     
Recommendation for user with id: 10
--------------------------------------

#1: I CAN'T GET STARTED By Ron Carter
#2: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) By Lonnie Gordon
#3: Just Dance By Lady GaGa / Colby O'Donis
#4: Home By Edward Sharpe & The Magnetic Zeros
#5: Unite (2009 Digital Remaster) By Beastie Boys
#6: West One (Shine On Me) By The Ruts
#7: Crossfire By Rick Cua
#8: Dans Ma Bulle (Edit Radio - Live 2006) By Diam's
#